<a href="https://colab.research.google.com/github/forexms78/AI-05-/blob/main/Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attention

Embedding + BiLSTM 모델과 추가로 Seq2seq을 구성하여 Attention 층을 추가

## 1. 감정 분류을 위한 Attention 순환 신경망: BiLSTM + Attention

기존 Embedding + BiLSTM 모델 아키텍처에 Bahdanau Attention(Additive)을 추가

BiLSTM 층을 통과해 나온 특성 벡터를 폴링 하지 않고 Attention을 통해 문장 전체에서 가장 중요한 토큰들의 정보가 압축된 컨텍스트 벡터(context vector)를 만듦


### 단계:

1. 모델 정의: BiLSTMWithAttention 클래스를 만듭니다.

2. 데이터 준비: AIHUB 감성 대화 데이터를 로드합니다.

3. 토크나이저: SentencePiece (BPE) 토크나이저를 학습시킵니다.

4. 데이터셋/로더: PyTorch Dataset 및 DataLoader를 구축합니다.

5. 학습 및 평가: 모델을 학습시키고, 손실 및 정확도 변화를 시각화하여 어텐션의 효과를 확인합니다.

### 모델링

**임베딩 레이어 사용**
- 토큰의 정수 인덱스로 부터 Embedding 레이어를 통 정수를 해당하는 임베딩 벡터로 매핑

**Bi-LSTM (Bidirectional LSTM) 레이어**
- 입력된 임베딩 벡터를 시간 축을 따라 처리하여, 시퀀스 데이터의 문맥 정보를 학습
- 양방향 LSTM은 입력 데이터를 과거와 미래 방향으로 모두 처리해 더 풍부한 문맥 정보를 캡처

**어텐션 적용(Bahdanau Attention)**
- 일반적인 LSTM은 마지막 타임스텝의 hidden state만을 사용하거나, 모든 hidden state를 단순 평균
- LSTM이 출력한 각 타임스텝의 hidden state를 평가하여, 시퀀스에서 중요한 단어(정보)에 더 높은 가중치를 부여
- 모델은 높은 가중치를 가진 중요한 단어에 집중하여 결과를 예측

- 어텐션 과정

$$u_t = \tanh(W_{\text{att}} \cdot h_t)$$


$$a_t = \text{softmax}(W_c \cdot u_t)$$

$$c = \sum_t a_t \cdot h_t$$


<center><img src="https://drive.google.com/uc?export=view&id=1oxXb1eh8ZMcIEgAvyMiw2B4TaPm81sl8" width="600"/></center>

    1. 선형 변환 이후 tanh 활성화를 통해 어텐션 스코어를 계산하기 위한 중간 표현(u)을 생성
    2. u를 단일 유닛으로 변환하여 각의 토큰 위치에 대한 스칼라 가중치(로짓)를 뽑아냄
    3. 로짓을 소프트맥스 활성화 하여 각 토큰 위치에 대한 확률 분포인 어텐션 가중치(a)를 생성
    5. 어텐션 가중치(a)는 한 문장 내에서 가장 중요한 토큰에는 높은 값이 할당됨
    6. 시퀀스 차원의 토큰 벡터 out에 어텐션 가중치 a를 각각 곱하고 가중합(Weighted Sum) 하여, 컨텍스트 벡터(Context Vecto)를 구함
    7. 산출된 컨텍스트 벡터에는 각 배치별로 문장 전체에서 가장 중요한 토큰들의 정보가 압축


In [8]:
import torch
import torch.nn as nn

# 감정 분류를 위한 순환 신경망 모델
'''
임베딩 레이어: 입력된 토큰을 임베딩 벡터로 변환합니다.
Bi-LSTM 레이어: 임베딩 벡터를 처리하여 문맥 정보를 학습합니다. 양방향 LSTM을 사용하여 과거와 미래의 문맥을 모두 고려합니다.
어텐션 메커니즘: Bi-LSTM의 출력 중 문장에서 중요한 부분에 더 집중하여 컨텍스트 벡터를 생성합니다. 이를 통해 문장 전체에서 가장 중요한 정보를 추출합니다.
'''
class BiLSTMWithAttention(nn.Module):

  def __init__(self, vocab_size, embedding_dim, hidden_size=12, num_layers=4, num_classes=2):

    super(BiLSTMWithAttention, self).__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_dim)

    self.lstm = nn.LSTM(
        input_size = embedding_dim,
        hidden_size = hidden_size,
        num_layers = num_layers,
        bidirectional = True,
        batch_first = True
    )

    self.attention = nn.Linear(hidden_size*2, hidden_size*2)
    self.context_vector = nn.Linear(hidden_size*2, 1, bias=False)

    self.layer_norm = nn.LayerNorm(hidden_size*2)
    self.dropout = nn.Dropout(p=0.2)

    #출력 레이어 2개
    self.fc1 = nn.Linear(hidden_size*2, hidden_size)
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):

    x = self.embedding(x)
    out, _ = self.lstm(x) # Changed from self.lstm(x) to self.lstm(embedded)
    print(f'lstm: {out.shape}')

    #덧셈으로하는 메커니즘
    u = torch.tanh(self.attention(out))
    print(f'u: {u.shape}')

    #softmax
    a = torch.softmax(self.context_vector(u), dim=1) # Changed from self.context_vecter(u) to self.context_vector(u)
    print(f'a: {a.shape}')

    context = (a * out).sum(dim=1) # Changed dim=10 to dim=1
    print(f'context: {context.shape}')

    context = self.layer_norm(context)
    context = self.dropout(context)
    dense = self.fc1(context)
    out = self.fc2(dense)

    return out

#모델 생성
model = BiLSTMWithAttention(vocab_size=100, embedding_dim=32)
input = torch.tensor([1,2,3,4]).unsqueeze(0) # Added unsqueeze(0) to add batch dimension
output = model(input)
print(f'모델 출력 결과 : {output}')

lstm: torch.Size([1, 4, 24])
u: torch.Size([1, 4, 24])
a: torch.Size([1, 4, 1])
context: torch.Size([1, 24])
모델 출력 결과 : tensor([[-0.1774,  0.1297]], grad_fn=<AddmmBackward0>)
